<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_1107softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ソフトマックス関数解題

- data: 2022_1107


In [ ]:
%config InlineBackend.figure_format = 'retina'
import numpy as np
import sys
import scipy
#from scipy.special import softmax
import matplotlib.pyplot as plt
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

def Softmax(x:np.array,
            beta:float=1.0):
    return np.exp(x/beta)/np.exp(x/beta).sum()

data = np.array(np.arange(5))[::-1]
betas = [0.1, 0.2, 0.4, 0.8,
         1, 2, 4, 8,
         16, 32, 64, 128]

plt.figure(figsize=(12,5))
for i, beta in enumerate(betas):
    data = np.array(np.arange(5))[::-1]
    output = Softmax(data, float(beta))
    plt.subplot(3,4,(i+1))
    plt.ylim(0,1)
    plt.bar(range(len(output)), output)
    plt.title(f'beta:{beta}')
    #print(output)
plt.show()


### シグモイド関数とソフトマックス関数の関係

ニューラルネットワークで用いられる出力関数には，シグモイド sigmoid 関数，ソフトマックス softmax 関数，がある。
本節では，この 2 つの関数に絞って，注意との関係を述べておくことにする。
ここで紹介する事項は，ほぼ高等学校初学年で程度の数学で理解できる事項であるため，冗長な記述である。
ソフトマックス関数と注意の関係について既知の読者は次節まで読み飛ばしていただきたい。

シグモイド関数，より正確にはロジスティック・シグモイド関数，の定義式は次式で与えられる:
$$
\sigma(x)=\left[1+\exp(-x)\right]^{-1}.
$$

もちろん表記の簡便さのためであるが $\exp$ を $e$ と表記しても良い。
$e$ は自然対数の底であり，およそ 2.718 である。
シグモイド関数は，ニューラルネットワークの出力関数として非線形変換の一つとして導入されるこが多い。
ロジスティック回帰でも用いられるとおり，確率比 (ロジット比) でもある。
このため，ある事象 $x$ が生起する確率 $p(x)$ と生起しない確率 $1-p(x)$ との比 $p\left[1-p\right]^{-1}$ で定義される。
すなわち

$$
\begin{aligned}
p  &= \frac{1}{1+e^{-x}}\\
1-p &= \frac{1+e^{-x}}{1+e^{-x}} - \frac{1}{1+e^{-x}} = \frac{e^{x}}{1+e^{x}}\\
\end{aligned}\tag{1}
$$

すなわち，ある事象 $x$ が起こる確率 $p(x)$ と起こらない確率 $1-p(x)$ との比 $p/(1-p)$ は
$$
\frac{p}{1-p}=\frac{\frac{1}{1+e^{-x}}}{\frac{e^{-x}}{1+e^{-x}}}=\frac{1}{e^{-x}}=e^{x}
$$

あるいは，上式を $x$ について解いて，ニューラルネットワークの出力関数であるシグモイド関数は，
ネットワークの出力が起こる確率と起こらない確率の比の対数と考えても良い。
すなわち上式を $x$ について解けば次式を得る:

$$
x = \log\left|\frac{p(x)}{1-p(x)}\right|
$$

すなわち出力 $x$ は $x$ の起こる確率と起こらない確率 $1-p(x)$ の比の対数である。

以上を，ソフトマックス関数との関連で検討する。
ソフトマックス関数は，複数個の候補から，一つを選ぶ確率として定義される。
$x_{i}$ の選択確率 $p(x_{i})$ は次式で与えられる:

$$
p(x_i)=\frac{e^{x_{i}}}{\sum_{j\in N}e^{x_{j}}},
$$

ここで，$j$ は全ての項目についてである。
選択肢が 2 つ，$x$ と $y$ であれば，以下のようになる:

$$
p(x) = \frac{e^{x}}{e^{x}+e^{y}}
$$

分子，分母を $e^{x}$ で割れば，次式を得る:

$$
p(x) = \frac{1}{1+e^{-(x-y)}}
$$

すなわち，シグモイド関数の引数 $x$ を 2 つの出力の差 $x-y$ で置き換えたものが，選択肢数 2 のソフトマックス関数である。
このことを拡張すれば，選択肢数 N のソフトマックス関数は，上のシグモイド関数と同様に次式のように書き換える事が可能である。

$$
p(x_i) = \frac{e^{x_i}}{\sum_{j\in N}e^{x_j}} = \frac{e^{x_i}}{e^{x_i}+\sum_{j\ne i}e^{x_{j}}}=\frac{e^{\text{正事例}}}{e^{正事例}+\sum e^{負事例}}
$$

上式を，シグモイド関数と同様に，分子分母を $e^{\text{正事例}}$ で割ることにすれば，次式を得る:

$$
p(x_i) = \frac{1}{1+e^{-\text{正事例}}\sum e^{負事例}} = \frac{1}{1+\sum e^{-\left(\text{正事例}-\text{負事例}\right)}}
$$

上式の意味は，シグモイド関数を多項目に拡張し，他の選択項目に比べて，任意の項目がどれ程選択されやすいかを示す指標とみなすことができる。
また，この式は，正事例と負事例との差を強調することを意味する。
このため，sentenceBERT (arXiv:1908.10084) の微調整 (fine-tuning) や，word2vec (arXiv:1369.4168) の負事例サンプリングで用いられる，目的関数，すなわち対比学習 contrastive learninng の目的関数となっている。

シグモイド関数の微分は，$d\sigma(x)/dx = x (1-x)$ であることはよく知られている。
一方で，ソフトマックス関数の場合には，多変数関数であるから偏微分を用いて $\partial s(x_i)/\partial x_j$ を計算することになる。
具体的には，次式のとおりである:

$$
\frac{\partial s(x_i)}{\partial x_j}=\left\{
\begin{aligned}
x_{i} (1-x_{i})  & \text{  if $i=j$}\\
-x_{i} x_{j}     & \text{  otherwise}
\end{aligned} = x_{i}(\delta_{ij}-x_{j})
\right.
$$

ここに，$\delta_{ij}$ は Kronecker のデルタである。
上式の $i=j$ のとおり，ソフトマックス関数はシグモイド関数の多変数拡張であり，対角要素 $i=j$ ではシグモイド関数の微分に一致することがわかる。

シグモイド関数は，出力関数としての役割以外に，シナプス後接続による，神経活動の修飾を行う役割が考えられる。
すなわちシナプス結合による情報伝達のゲートとして役割が考えられる。

### ワンホットベクトルとソフトマックス

出力層のニューロンが複数個ある場合には，その一つを選択することが分類問題では求められる。
これは，ソフトマックス関数を用いて，最大値を与える選択肢の強調する。
すなわち，最大値をとる選択肢の選択確率を大きくすることを意味する。
逆に，最大値以外の値をとる他の選択肢の選択確率を低めることになる。

このような理由から，分類課題を解く場合，多層ニューラルネットワークの最終層では，ソフトマックス関数を用いることが多い。
多肢選択課題において，正解選択肢を選ぶ確率を与えると考えても良い。

出力層の出力において，最大値を与える選択肢の選択確率を高めることは，一つの選択肢の選択確率を 1 に近づけ，かつ，他の選択肢の選択確率を 0 に近づけることとなる。
この意味で，ワンホット表現 one-hot representation を得るために使われる。

著者の知る限り，ワンホット表現という用語が定着する以前では，one of k 表現と呼ばれることが多かった。
例えば Bishop の教科書 [@2006BishopPRML] (邦訳:パターン認識と機械学習) では one-of-k と記載されている。


### 双曲線正接関数 (ハイパータンジェント $\tanh$) との関係

シグモイド関数は，字義どおりの意味では，S 字状の曲線，という程度の意味である。
S 字状であるとは，単調非減少 $x< y\mapsto f(x)\le f(y)$ であることを意味する。
したがって，シグモイド関数と同じ定義域を持つ S 字状の曲線を持つ出力関数は，他にも考えられる。
例えば，正規分布の累積密度関数，整流線型化関数 ReLU, ハイパーボリックタンジェント $\tanh$ などが挙げられよう。

このうち，ハイパーボリックタンジェント，すなわち双曲線正接は $\tanh$ と表記されることが多い。
定義は次式のとおりである:

$$
\tanh(x) = \frac{e^{x}-e^{-x}}{e^{x}+e^{x}}\tag{eq:def_tanh}
$$

$\lim_{x\rightarrow-\infty}\tanh(x)=-1$，$\lim_{x\rightarrow\infty}\tanh(x)=1$ であるので，シグモイド関数とは値域が異なる。

LeCun による指摘が最初だと思われるが，$\tanh(x)$ を出力関数として用いることで，シグモイド関数を出力関数としても用いた場合と比較して，圧倒的に収束が速くなる。
収束が速くなる理由は 2 つ挙げられる。

1. 勾配消失問題 gradient vanishing problem が生じ難い。
2. シグモイド関数が持つバイアスを回避できる

以下，上記 2 項を簡単に解説する。

勾配消失問題とは，多層ニューラルネットワークにおいて，層を重ねると学習時に学習が難しいという指摘である。
ニューラルネットワークの学習は，一般に次式で定義される:

$$
\Delta\theta = -\eta\frac{\partial\ell}{\partial\theta}
$$

ここで $\theta$ はネットワークを定義するパラメータ，すなわち結合係数やバイアス項を含めたパラメータ，$\eta$ は学習係数と呼ばれるハイパーパラメータであるとする。
$\ell$ は，自乗誤差を含む誤差関数，目的関数，損失関数，など分野や流儀によって様々に称されている関数である。
本稿では，特に区別せず損失関数 loss function と呼ぶこととする。

この損失関数を最小化することをニューラルネットワークの学習と呼ぶ。

ここで，多層ニューラルネットワークの場合，合成関数の微分則，あるいはチェインルール chain rule を用いて出力層から入力層に向かって誤差を計算していくことが必要となる。
これを，誤差逆伝播 back propagation と呼ぶ。
シュミットフーバー Schmithuber [@2015Schmidhuber_review] によれば，誤差逆伝播法の提案は 1950 年代あるいはそれ以前に遡ることができる。
チェインルールを言葉で書けば，$f$ が $x$ の関数であり，$x$ が $y$ の関数であれば，$f$ を $y$ で微分する際に，$f$ を $x$ で微分し，然る後に $x$ を $y$ で微分すれば良い，ということになる。
極めて単純な話だが，これが多層ニューラルネットワークの層ごとに行われると捉えれば良い。

勾配消失問題に戻ると，層を遡及する，すなわち逆伝播する際に，かならずある関数の微分が掛けられることを意味する。
したがって，損失関数の値が大きな値であっても，層を逆行するたびに，微係数が掛けられるので，出力関数の微係数が小さければ，入力層に近い下層では誤差が指数関数的に小さくなることを意味する。
微係数が小さいことは，収束が遅くなることを意味する。
ことのため，微係数が小さい出力関数では学習が遅くなる。
これを称して，勾配消失問題と言う。

実際，シグモイド関数の微係数は $\sigma'(x)=\sigma(x)(1 -\sigma(x))$ である。
したがって，微係数，すなわち接戦の傾きが最大となるのは $x=0$ のときであり，$\sigma'(x=0) = 0.5 (1-0.5)=0.25$ となる。
一方，ハイパータンジェントの場合には $\tanh'(x)= 1 - (\tanh(x))^2$ である。
$\tanh'(x)$ の最大値も $x=0$ のときであり，このときの値は 1 である。
層を重ねるに従って，この効果は拡大する。


## 分配関数との関係

ソフトマックス関数，あるいは，その特別な場合としてのシグモイド関数の分母は $\sum_{i\in{X}}e^{x_{i}}$ は，統計力学の分配関数と数式の表記上は同一である。
より具体的にに，ある系のすべての状態の集合を $\Omega$ とし，系の状態を $\omega\in\Omega$ とすれば次式で与えられる:

$$
Z(\beta) = \sum_{\omega\in\Omega}e^{-\beta{E}(\omega)}
$$

和の中の $\displaystyle e^{-\beta {\mathcal{E}}(\omega)}$ はボルツマン因子と呼ばれる。
カノニカルアンサンブルは熱浴と接触する閉鎖系を表現するアンサンブルである。
パラメータ $\beta$ は熱浴を特徴づける量で，熱浴の温度と解釈される。
熱力学温度 T とは $\beta=1/kT$ の関係にあり，逆温度と呼ばれる。
$k$ はボルツマン定数である。
分配関数に定数を乗じることはエネルギーの基準値をずらすことに等しい。
分配関数の大きさそのものには意味がない。

$\beta$ については，蒸留 distillation について説明する項で説明する。

## 交差エントロピー cross entropoy との関係

交差エントロピー自体は，1980 年代から知られていた目的関数であり，2 値を取る教師信号に対して，収束が早い望ましい目的関数と考えれる。
教師信号を $t$，モデルからの出力を $p$ とすれば，次式:

$$
\ell_{CE}=-t\log(p) + (1-t)\log(1-p)
$$

で与えれる [@1989Hinton]。
上式右辺は，$t\in\{0,1\}$ の 2 値の教師信号を取るため，$t=1$ の場合には右辺第一項が選ばれる。
一方，このとき右辺第二項は 0 となるため考慮されない。
反対に $t=0$ であれば，右辺第一項は 0 となり無視され，反対に右辺第二項のみが有効となる。
このため交差エントロピー損失は，プログラミングコードにおいて `if` 文を使ったような効果が認められる。

平均自乗誤差に比して，教師信号 t と出力 p との乖離が多くくなるにつれて値が急激に大きくなるため，
学習が高速化される。
以下の Python コードを実行して交差エントロピー関数の意味を確認して欲しい。

<div class="code">

```python
%config InlineBackend.figure_format = 'retina'   # Mac retina display でのおまじない
import matplotlib.pyplot as plt                  # グラフ描画ライブラリを輸入する
import numpy as np                               # Numpy の輸入

X = np.linspace(0.001, .999)                     # 無限大になることを避けるために [0,1] ではなく (0,1) の範囲にする
ce = lambda x: - np.log(x)                       # 交差エントロピーの定義

plt.plot(X, ce(X),   label='ce0')                # 実際の描画命令
plt.plot(X, ce(1-X), label='ce1')                # 同上
plt.legend()                                     # 凡例の描画
plt.title('Plot of cross-entropy loss function') # 図のタイトル
plt.show()                                       # 画面表示
```
</div>

交差エントロピーの定義式に若干の変形を加えれば，次式の如くなるため，シグモイド関数との関係は明白である。
$$
\ell_{CE} = -\log\frac{p^{t}}{(1-p)^{1-t}}
$$

したがって，シグモイド関数，あるいは，その拡張であるソフトマックス関数を用いる際に，交差エントロピーを用いることにすれば，
実際の収束が速いのみならず，情報量最大化，すなわちエントロピー最大化を満たす最尤解を得ることに相当する。

また，tSNE や VAE で，符号化器と復号化器との間での KL divergence との関係も指摘できる

エントロピーについては，熱力学と情報論とで用いられる概念であって，ニューラルネットワークでは，熱力学的にも情報論的にも用いられる。
両者とも $-p\log p$ で定義される。
情報論的には，エントロピーの最大化，あるいは情報量最大化という形で定義される。
すなわちニューラルネットワークの損失関数として用いられる。

したがって交差エントロピーとは，情報量最大化であり，ニューラルネットワークの出力がワンホットベクトルとして与えられる場合，
最終層をソフトマックス関数として定義し，かつ，損失関数を交差エントロピーで定義すれば，エントロピーの最大化，すなわち情報量最大化にもなっていることを意味する。


In [ ]:
import numpy as np
np.set_printoptions(precision=5)

def kWTA(X:np.ndarray=np.random.randn(5), # 入力層
         k:int=1,                         # 勝者の数
         alpha = 0.,                      # 崩壊率
         _lambda:float=1.0,               # 掃除層からのフィードバックかかるハイパーパラメータ
                                          # lambda は Python の予約語で使えないため _labmda にした
         n2k_limit:float=0.000001,
         n2k_threshold:float=1.0,
         max_t:int=50000,
         verbose=False)->dict:
    """WTA の実装 """

    max_u = np.argmax(X) # 初期状態での最大活性値を与えるユニット番号を保持しておく
    if alpha == 0:       # 崩壊率，ユニット数が多いほど，小さい数値にした方が安定するので，こうしてみた。
        alpha = 10 ** -(np.log10(X.shape[0])+5)

    n = float(len(X))
    n2k = n - (2. * k)

    for t in range(max_t):                # max_t を上限として反復
        Y = np.tanh(X)                    # 出力の計算
        n_winners = ((Y > 0) * 1.).sum()  # 勝者数，出力が正のユニット数を計測
        if ((n_winners == k)):            # 勝者数が条件どおりなら終了
            break

        old_n2k = n2k                     # 内部状態の更新，すなわち，拘束条件の計算
        n2k = np.sum(Y) + (n - (2. * k))
        #if np.abs(n2k - old_n2k) < n2k_limit: break

        X += - alpha * Y - _lambda * n2k  # 右辺第 1 項は自己回帰，右辺第 2 項は掃除層からのフィードバック

        if verbose:
            print(f't:{t} n_winners:{n_winners}, n2k:{n2k:.3f} old_n2k:{old_n2k:.3f}')
            print(f'X:{X}\n   X:{Y}')

    _max_u = np.argmax(Y)                  # 反復計算終了時の最大活性を示すユニット番号
    OK = False if max_u != _max_u or t == max_t else True

    return {'Y':Y, 'n_winners':n_winners, 'n2k':n2k, 'time':t, 'OK':OK}

In [ ]:
import numpy as np
import sys
import scipy
from scipy.special import softmax
import matplotlib.pyplot as plt
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

def Softmax(x:np.array,
            beta:float=1.0,
           ):
    d0 = np.exp(x/beta)
    d1 = np.exp(x/beta).sum()
    return d0/d1

data = np.array(np.arange(5))[::-1]
#print(f'data:{data}')
betas = [ 4,  8, 12, 14, 15, 16,
         20, 24, 25, 26, 27, 28,
         29, 30, 31, 32, 33, 38,
         42, 48, 52, 58, 60, 64]

plt.figure(figsize=(20,14))
for i, beta in enumerate(betas):
    data = np.array(np.arange(5))[::-1]
    output = Softmax(data, float(beta))
    plt.subplot(4, 6,(i+1))
    plt.ylim(0,1)
    plt.bar(range(len(output)), output)
    #print(output)
    ret = kWTA(X=output, k=1)
    plt.title(f'beta:{beta}, 反復回数:{ret["time"]}')
plt.show()
